<a href="https://colab.research.google.com/github/Sciederrick/Data-Science-For-EveryOne/blob/main/rfm_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RFM ANALYSIS

- RFM analysis is a powerful technique used by companies to better understand customer behaviour and optimize engagement strategies. It revolves around three key dimensions: recency, frequency, and monetary value. These dimensions capture essential aspects of customer transactions, providing valuable information for segmentation and personalized marketing campaigns.

- The given dataset is provided by an e-commerce platform containing customer transaction data including customer ID, purchase date, transaction amount, product information, ID command and location. The platform aims to leverage RFM (recency, frequency, monetary value) analysis to segment customers and optimize customer engagement strategies.

- Your task is to perform RFM analysis and develop customer segments based on their RFM scores. The analysis should provide insights into customer behaviour and identification of high-value customers, at-risk customers, and potential opportunities for personalized marketing campaigns.

In [175]:
# Suppress all warnings
import warnings
warnings.filterwarnings("ignore")


In [176]:
import pandas as pd
# The following lines adjust the granularity of reporting.
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format

In [177]:
file_id = "1DsgZ316R73v4vDZ0UZjHlwbWq0_-NqvR"
url = 'https://drive.google.com/uc?id={}'.format(file_id)
df = pd.read_csv(url)
df.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location
0,8814,2023-04-11,943.3,Product C,890075,Tokyo
1,2188,2023-04-11,463.7,Product A,176819,London
2,4608,2023-04-11,80.3,Product A,340062,New York
3,2559,2023-04-11,221.3,Product A,239145,London
4,9482,2023-04-11,739.6,Product A,194545,Paris


In [178]:
df.shape

(1000, 6)

In [179]:
df.describe()

,CustomerID,TransactionAmount,OrderID
count,1000.0,1000.0,1000.0
mean,5554.8,513.7,554071.4
std,2605.0,286.1,264695.4
min,1011.0,12.1,100096.0
25%,3273.0,257.1,313152.0
50%,5538.0,523.6,564671.5
75%,7821.8,759.9,783052.2
max,9991.0,999.4,999695.0


In [180]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CustomerID          1000 non-null   int64  
 1   PurchaseDate        1000 non-null   object 
 2   TransactionAmount   1000 non-null   float64
 3   ProductInformation  1000 non-null   object 
 4   OrderID             1000 non-null   int64  
 5   Location            1000 non-null   object 
dtypes: float64(1), int64(2), object(3)
memory usage: 47.0+ KB


In [181]:
df.isnull().sum()

CustomerID            0
PurchaseDate          0
TransactionAmount     0
ProductInformation    0
OrderID               0
Location              0
dtype: int64

In [182]:
df['PurchaseDate'] = df['PurchaseDate'].astype('datetime64')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   CustomerID          1000 non-null   int64         
 1   PurchaseDate        1000 non-null   datetime64[ns]
 2   TransactionAmount   1000 non-null   float64       
 3   ProductInformation  1000 non-null   object        
 4   OrderID             1000 non-null   int64         
 5   Location            1000 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 47.0+ KB


In [183]:
# @title Recency
from datetime import datetime
df['Recency'] = (datetime.now().date() - df['PurchaseDate'].dt.date).dt.days
df.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency
0,8814,2023-04-11,943.3,Product C,890075,Tokyo,185
1,2188,2023-04-11,463.7,Product A,176819,London,185
2,4608,2023-04-11,80.3,Product A,340062,New York,185
3,2559,2023-04-11,221.3,Product A,239145,London,185
4,9482,2023-04-11,739.6,Product A,194545,Paris,185


In [184]:
# @title Frequency
df_frequency_agg = df.groupby('CustomerID').agg(Frequency=('OrderID', 'count')).reset_index()
df_frequency_agg.head()

,CustomerID,Frequency
0,1011,2
1,1025,1
2,1029,1
3,1046,1
4,1049,1


In [185]:
df_frequency_agg.count() # unique rows

CustomerID    946
Frequency     946
dtype: int64

In [186]:
df = df.merge(df_frequency_agg, on='CustomerID', how='left')
df.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency
0,8814,2023-04-11,943.3,Product C,890075,Tokyo,185,1
1,2188,2023-04-11,463.7,Product A,176819,London,185,1
2,4608,2023-04-11,80.3,Product A,340062,New York,185,1
3,2559,2023-04-11,221.3,Product A,239145,London,185,1
4,9482,2023-04-11,739.6,Product A,194545,Paris,185,1


In [187]:
# @title Monetary Value
df_monetary_agg = df.groupby('CustomerID').agg(MonetaryValue=('TransactionAmount', 'sum')).reset_index()
df_monetary_agg.head()

,CustomerID,MonetaryValue
0,1011,1129.0
1,1025,359.3
2,1029,705.0
3,1046,859.8
4,1049,225.7


In [188]:
df_monetary_agg.count() # Unique rows

CustomerID       946
MonetaryValue    946
dtype: int64

In [189]:
df = df.merge(df_monetary_agg, on='CustomerID', how='left')
df.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency,MonetaryValue
0,8814,2023-04-11,943.3,Product C,890075,Tokyo,185,1,943.3
1,2188,2023-04-11,463.7,Product A,176819,London,185,1,463.7
2,4608,2023-04-11,80.3,Product A,340062,New York,185,1,80.3
3,2559,2023-04-11,221.3,Product A,239145,London,185,1,221.3
4,9482,2023-04-11,739.6,Product A,194545,Paris,185,1,739.6


In [190]:
df.describe()

,CustomerID,TransactionAmount,OrderID,Recency,Frequency,MonetaryValue
count,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
mean,5554.8,513.7,554071.4,155.7,1.1,574.2
std,2605.0,286.1,264695.4,17.3,0.3,361.6
min,1011.0,12.1,100096.0,125.0,1.0,12.1
25%,3273.0,257.1,313152.0,140.0,1.0,283.9
50%,5538.0,523.6,564671.5,157.0,1.0,566.7
75%,7821.8,759.9,783052.2,170.0,1.0,805.4
max,9991.0,999.4,999695.0,185.0,3.0,2379.4


In [191]:
# @title Recency Score
recency_scores = [5, 4, 3, 2, 1]
df['RecencyScore'] = pd.cut(df['Recency'], bins=5, labels=recency_scores)
df['RecencyScore'] = df['RecencyScore'].astype(int)
df.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency,MonetaryValue,RecencyScore
0,8814,2023-04-11,943.3,Product C,890075,Tokyo,185,1,943.3,1
1,2188,2023-04-11,463.7,Product A,176819,London,185,1,463.7,1
2,4608,2023-04-11,80.3,Product A,340062,New York,185,1,80.3,1
3,2559,2023-04-11,221.3,Product A,239145,London,185,1,221.3,1
4,9482,2023-04-11,739.6,Product A,194545,Paris,185,1,739.6,1


In [192]:
# @title Frequency Score
frequency_scores = [1, 2, 3, 4, 5]
df['FrequencyScore'] = pd.cut(df['Frequency'], bins=5, labels=frequency_scores)
df['FrequencyScore'] = df['FrequencyScore'].astype(int)
df.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency,MonetaryValue,RecencyScore,FrequencyScore
0,8814,2023-04-11,943.3,Product C,890075,Tokyo,185,1,943.3,1,1
1,2188,2023-04-11,463.7,Product A,176819,London,185,1,463.7,1,1
2,4608,2023-04-11,80.3,Product A,340062,New York,185,1,80.3,1,1
3,2559,2023-04-11,221.3,Product A,239145,London,185,1,221.3,1,1
4,9482,2023-04-11,739.6,Product A,194545,Paris,185,1,739.6,1,1


In [193]:
# @title Monetary Score
monetary_scores = [1, 2, 3, 4, 5]
df['MonetaryScore'] = pd.cut(df['MonetaryValue'], bins=5, labels=monetary_scores)
df['MonetaryScore'] = df['MonetaryScore'].astype(int)
df.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency,MonetaryValue,RecencyScore,FrequencyScore,MonetaryScore
0,8814,2023-04-11,943.3,Product C,890075,Tokyo,185,1,943.3,1,1,2
1,2188,2023-04-11,463.7,Product A,176819,London,185,1,463.7,1,1,1
2,4608,2023-04-11,80.3,Product A,340062,New York,185,1,80.3,1,1,1
3,2559,2023-04-11,221.3,Product A,239145,London,185,1,221.3,1,1,1
4,9482,2023-04-11,739.6,Product A,194545,Paris,185,1,739.6,1,1,2


In [194]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   CustomerID          1000 non-null   int64         
 1   PurchaseDate        1000 non-null   datetime64[ns]
 2   TransactionAmount   1000 non-null   float64       
 3   ProductInformation  1000 non-null   object        
 4   OrderID             1000 non-null   int64         
 5   Location            1000 non-null   object        
 6   Recency             1000 non-null   int64         
 7   Frequency           1000 non-null   int64         
 8   MonetaryValue       1000 non-null   float64       
 9   RecencyScore        1000 non-null   int64         
 10  FrequencyScore      1000 non-null   int64         
 11  MonetaryScore       1000 non-null   int64         
dtypes: datetime64[ns](1), float64(2), int64(7), object(2)
memory usage: 101.6+ KB


In [195]:
# @title RFM Score
df['RFMScore'] = df['RecencyScore'] + df['FrequencyScore'] + df['MonetaryScore']
df.describe()

,CustomerID,TransactionAmount,OrderID,Recency,Frequency,MonetaryValue,RecencyScore,FrequencyScore,MonetaryScore,RFMScore
count,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
mean,5554.8,513.7,554071.4,155.7,1.1,574.2,3.0,1.2,1.7,5.9
std,2605.0,286.1,264695.4,17.3,0.3,361.6,1.4,0.7,0.7,1.8
min,1011.0,12.1,100096.0,125.0,1.0,12.1,1.0,1.0,1.0,3.0
25%,3273.0,257.1,313152.0,140.0,1.0,283.9,2.0,1.0,1.0,4.0
50%,5538.0,523.6,564671.5,157.0,1.0,566.7,3.0,1.0,2.0,6.0
75%,7821.8,759.9,783052.2,170.0,1.0,805.4,4.0,1.0,2.0,7.0
max,9991.0,999.4,999695.0,185.0,3.0,2379.4,5.0,5.0,5.0,15.0


In [199]:
# Develop customer segments based on their RFM scores
segment_labels = ['Low', 'Medium', 'High']
df['ValueSegment'] = pd.cut(df['RFMScore'], bins=3, labels=segment_labels)
df.describe()

,CustomerID,TransactionAmount,OrderID,Recency,Frequency,MonetaryValue,RecencyScore,FrequencyScore,MonetaryScore,RFMScore
count,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
mean,5554.8,513.7,554071.4,155.7,1.1,574.2,3.0,1.2,1.7,5.9
std,2605.0,286.1,264695.4,17.3,0.3,361.6,1.4,0.7,0.7,1.8
min,1011.0,12.1,100096.0,125.0,1.0,12.1,1.0,1.0,1.0,3.0
25%,3273.0,257.1,313152.0,140.0,1.0,283.9,2.0,1.0,1.0,4.0
50%,5538.0,523.6,564671.5,157.0,1.0,566.7,3.0,1.0,2.0,6.0
75%,7821.8,759.9,783052.2,170.0,1.0,805.4,4.0,1.0,2.0,7.0
max,9991.0,999.4,999695.0,185.0,3.0,2379.4,5.0,5.0,5.0,15.0


In [201]:
df_for_corr = df.drop(columns=['CustomerID'])
df_for_corr.corr()

,TransactionAmount,OrderID,Recency,Frequency,MonetaryValue,RecencyScore,FrequencyScore,MonetaryScore,RFMScore
TransactionAmount,1.0,0.0,-0.0,0.0,0.8,0.0,0.0,0.7,0.3
OrderID,0.0,1.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0
Recency,-0.0,-0.0,1.0,0.1,0.0,-1.0,0.1,0.0,-0.7
Frequency,0.0,0.0,0.1,1.0,0.5,-0.1,1.0,0.5,0.6
MonetaryValue,0.8,0.0,0.0,0.5,1.0,-0.0,0.5,0.9,0.6
RecencyScore,0.0,0.0,-1.0,-0.1,-0.0,1.0,-0.1,-0.0,0.7
FrequencyScore,0.0,0.0,0.1,1.0,0.5,-0.1,1.0,0.5,0.6
MonetaryScore,0.7,0.0,0.0,0.5,0.9,-0.0,0.5,1.0,0.6
RFMScore,0.3,0.0,-0.7,0.6,0.6,0.7,0.6,0.6,1.0


In [202]:
df.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency,MonetaryValue,RecencyScore,FrequencyScore,MonetaryScore,RFMScore,ValueSegment
0,8814,2023-04-11,943.3,Product C,890075,Tokyo,185,1,943.3,1,1,2,4,Low
1,2188,2023-04-11,463.7,Product A,176819,London,185,1,463.7,1,1,1,3,Low
2,4608,2023-04-11,80.3,Product A,340062,New York,185,1,80.3,1,1,1,3,Low
3,2559,2023-04-11,221.3,Product A,239145,London,185,1,221.3,1,1,1,3,Low
4,9482,2023-04-11,739.6,Product A,194545,Paris,185,1,739.6,1,1,2,4,Low
